# Install Taipy

To install Taipy, just `pip install` it.

In [1]:
%pip install taipy -q


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pmdarima -q


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip list

Package                  Version
------------------------ -----------
aiofiles                 22.1.0
aiosqlite                0.18.0
alembic                  1.10.2
aniso8601                9.0.1
anyio                    3.6.2
apispec                  5.2.2
apispec-webframeworks    0.5.2
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
asttokens                2.2.1
attrs                    22.2.0
Babel                    2.12.1
backcall                 0.2.0
beautifulsoup4           4.12.0
bidict                   0.22.1
bleach                   6.0.0
certifi                  2022.12.7
cffi                     1.15.1
charset-normalizer       3.1.0
click                    8.1.3
cloudpickle              2.2.1
comm                     0.1.3
Cython                   0.29.33
dask                     2023.3.1
debugpy                  1.6.6
decorator                5.1.1
deepdiff                 6.2.3
defusedxml               0.7.1
distributed 

# Import the packages

In [4]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config, Scope
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

# Taipy Gui Basics
## Markdown Syntax

Taipy uses the Markdown syntax to display elements. `#` creates a title, `*` puts your text in italics and `**` puts it in bold.

![](img/gui_basic_eng.png)

In [5]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [6]:
Gui(page_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:46,753][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:46,883][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Visual elements
Create different visual elements. The syntax is always the same for each visual element.  `<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- Create a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) `<|{value}|slider|>`

- Create a [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) `<|{value}|date|>`

- Create a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) `<|{value}|selector|lov={list_of_values}|>`


![](img/control.png)

In [7]:
slider_value = 0
date_value = None
selected_value = None
selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|{date_value}|date|> <|{date_value}|>

<|{selected_value}|selector|lov={selector}|> <|{selected_value}|>
"""

In [8]:
Gui(control_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:46,913][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:46,961][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:46,962][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Data Viz

A dataset gathering information on the number of deaths, confirmed cases and recovered for different regions is going to be used to create an interactive Dashboard.

In [9]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [10]:
def initialize_case_evolution(data, selected_country='India') -> pd.DataFrame:
    # Aggregation of the dataframe per Country/Region
    country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()
    
    # a country is selected, here India by default
    country_date_df = country_date_df.loc[country_date_df['Country/Region']==selected_country]
    return country_date_df

In [11]:
country_date_df = initialize_case_evolution(data)
country_date_df.head()

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
23021,India,2020-01-30,23.746783,78.96288,1.0,0.0,0.0
23022,India,2020-01-31,23.746783,78.96288,1.0,0.0,0.0
23023,India,2020-02-01,23.746783,78.96288,1.0,0.0,0.0
23024,India,2020-02-02,23.746783,78.96288,2.0,0.0,0.0
23025,India,2020-02-03,23.746783,78.96288,3.0,0.0,0.0


Create a [chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) showing the evolution of Deaths in France (_Deaths_ for _y_ and _Date_ for _x_). The visual element (chart) has the same syntax as the other ones with specific properties (_x_, _y_, _type_ for example). Here are some [examples of charts](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/). The _x_ and _y_ porperties only needs the name of the dataframe columns to display.

![](img/simple_graph.png)

In [12]:
country_md = "<|{country_date_df}|chart|x=Date|y=Deaths|type=bar|>"

In [13]:
Gui(country_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:47,770][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:47,819][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:47,820][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Add new traces

- Add on the graph the number of Confirmed and Recovered cases (_Confirmed_ and _Recovered_) with the number of Deaths
- _y_ (and _x_) can be indexed this way to add more traces (`y[1]=`, `y[2]=`, `y[3]=`).

![](img/multi_traces.png)

In [14]:
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|>"

In [15]:
Gui(country_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:47,840][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:47,889][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:47,890][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Style the graph with personalized properties
The _layout_ dictionnary specifies how bars should be displayed. They would be 'stacked'.

These are Plotly properties.

![](img/stack_chart.png)

In [16]:
layout = {'barmode':'stack'}
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>"

In [17]:
Gui(country_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:47,911][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:47,963][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:47,964][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Add texts that sums up the data

Use the [text](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) visual element.

- Add the total number of Deaths (last line of _country_date_df_)
- Add the total number of Recovered (last line of _country_date_df_)
- Add the total number of Confirmed (last line of _country_date_df_)


In [18]:
country_date_df

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
23021,India,2020-01-30,23.746783,78.962880,1.0,0.0,0.0
23022,India,2020-01-31,23.746783,78.962880,1.0,0.0,0.0
23023,India,2020-02-01,23.746783,78.962880,1.0,0.0,0.0
23024,India,2020-02-02,23.746783,78.962880,2.0,0.0,0.0
23025,India,2020-02-03,23.746783,78.962880,3.0,0.0,0.0
...,...,...,...,...,...,...,...
23353,India,2020-12-27,854.924665,3023.983447,10207871.0,9782669.0,147901.0
23354,India,2020-12-28,854.924665,3023.983447,10224303.0,9807569.0,148153.0
23355,India,2020-12-29,854.924665,3023.983447,10244852.0,9834141.0,148439.0
23356,India,2020-12-30,854.924665,3023.983447,10266674.0,9860280.0,148738.0


This is how we can get the total number of Deaths from the dataset for India.

In [19]:
country_date_df.iloc[-1, 6] # gives the number of deaths for India (5 is for recovered and 4 is confirmed)

148738.0

Use the [text](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) visual element. Note that between `{}`, any Python variable can be put but also any Python code.

![](img/control_text.png)

In [20]:
country_md = """
## Deaths <|{country_date_df.iloc[-1, 6]}|text|>

## Recovered <|{country_date_df.iloc[-1, 5]}|text|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|text|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>
"""

In [21]:
Gui(country_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:48,009][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:48,063][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:48,064][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Local _on_change_

- Add a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) with `dropdown=True` containing the name of all the _Country/region_
- Give to the _on_change_ selector property the name of the _on_change_country_ function. This function will be called when the selector will be used.
- This function has a 'state' parameter and has to be completed. When the selector is used, this function is called with the _state_ argument. It contains all the Gui variables; 'state.country_date_df' is then the dataframe used in the Gui.

![](img/on_change_local.png)

In [22]:
country_lov = sorted(data["Country/Region"].dropna().unique().tolist())
selected_country = "India"

country_md = """
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>
"""

In [23]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # state.selected_country, state.country_date_df, ...
    # update country_date_df with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    state.country_date_df = initialize_case_evolution(data, state.selected_country)

In [24]:
Gui(country_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:48,157][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:48,207][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:48,208][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


## Layout

Use the [layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/) block to change the page structure. This block creates invisible columns to put text/visual elements in.

Syntax :
```
<|layout|columns=1 1 1 ...|
(first column)

(in second column)

(third column)
(again, third column)

(...)
|>
```

In [25]:
final_country_md = """
<|layout|columns=1 1 1 1|
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>
|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|>
"""

In [26]:
Gui(final_country_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:48,232][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:48,287][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:48,288][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


![](img/layout.png)

# Map

In [27]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths in ' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [28]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths in Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths in Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths in Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths in Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths in Angola


Properties to style the map
- marker color corresponds to the number of Deaths (column _Deaths_)
- marker sizes corresponds to the size in _Size_ column which is created from the number of Deaths

layout_map permet defined the initial zoom and position of the map


In [29]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

We give to Plotly:
- a map type
- the name of the latitude column
- the name of the longitude column
- properties: on the size and color of the markers
- the name of the column for the text of the points

In [30]:
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|>
"""

In [31]:
Gui(map_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:49,011][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:49,062][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:49,062][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


![](img/carte.png)

# Part and the _render_ property
- Create a [toggle](https://docs.taipy.io/en/latest/manuals/gui/viselements/toggle/) (works the same as a selector) with a lov of 'Map' an 'Country'
- Create two part blocks that renders or not depending on the value of the toggle
    - To do this, use the fact that in the _render_ property of the part block, Python code can be inserted in `{}`

In [32]:
representation_toggle = ['Map', 'Country']
selected_representation = representation_toggle[0]

In [33]:
main_page = """
<|{selected_representation}|toggle|lov={representation_toggle}|>

<|part|render={selected_representation == "Country"}|
"""+final_country_md+"""
|>

<|part|render={selected_representation == "Map"}|
"""+map_md+"""
|>
""" 

In [34]:
Gui(main_page).run(dark_mode=False, run_browser=False)

[2023-03-24 22:51:49,090][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:51:49,142][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:51:49,142][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


![](img/part_render.png)

# Taipy Core
Here are the functions that we are going to use to predict the number of Deaths for a country.
We will:
- preprocess the data (_preprocess_),
- create a training and testing database (_make_train_test_data_),
- train a model (_train_model_),
- generate predictions (_forecast_),
- generate a dataframe with the historical data and the predictions (_result_)

![](img/all_architecture.svg)

In [35]:
# initialise variables
selected_scenario = None
scenario_selector = None

first_date = dt.datetime(2020,11,1)

scenario_name = None

result = None

In [36]:
#Config.configure_job_executions(mode="standalone", nb_of_workers=2)

In [37]:

def add_features(data):
    dates = pd.to_datetime(data["Date"])
    data["Months"] = dates.dt.month
    data["Days"] = dates.dt.isocalendar().day
    data["Week"] = dates.dt.isocalendar().week
    data["Day of week"] = dates.dt.dayofweek
    return data

def create_train_data(final_data, date):
    bool_index = pd.to_datetime(final_data['Date']) <= date
    train_data = final_data[bool_index]
    return train_data

def preprocess(initial_data, country, date):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    final_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    final_data = final_data[['Date','Deaths']]
    final_data = add_features(final_data)
    
    train_data = create_train_data(final_data, date)
    return final_data, train_data


def train_arima(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model):
    predictions = model.predict(n_periods=60)
    return np.array(predictions)


def result(final_data, predictions, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(len(predictions))])
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    predictions = pd.concat([pd.Series(dates, name="Date"),
                             pd.Series(predictions, name="Predictions")], axis=1)
    return final_data.merge(predictions, on="Date", how="outer")


def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return predictions

First we must define the Data Nodes then the tasks (associated to the Python function). Furthermore, we gather these tasks into different pipelines and these pipelines into a scenario.

A Data Node needs a **unique id**. If needed, the storage type can be changed for CSV and SQL. Other parameters are then needed.

### Data Nodes and Task for preprocess

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [38]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5),
                                              scope=Scope.GLOBAL)

country_cfg = Config.configure_data_node(id="country", default_data="India", cacheable=True, validity_period=dt.timedelta(days=5))


date_cfg = Config.configure_data_node(id="date", default_data=dt.datetime(2020,10,10), cacheable=True, validity_period=dt.timedelta(days=5))

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [39]:
final_data_cfg =  Config.configure_data_node(id="final_data", cacheable=True, validity_period=dt.timedelta(days=5))

train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))


<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [40]:
task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg, date_cfg],
                                           output=[final_data_cfg,train_data_cfg])

### Data Nodes and Task for train_model

<img src="img/train_model.svg" alt="drawing" width="500"/>

In [41]:
model_cfg = Config.configure_data_node(id="model", scope=Scope.PIPELINE, cacheable=True, validity_period=dt.timedelta(days=5))

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_arima,
                                      input=train_data_cfg,
                                      output=model_cfg)

### Data Nodes and Task for forecast

<img src="img/forecast_arima.svg" alt="drawing" width="500"/>

In [42]:
predictions_cfg = Config.configure_data_node(id="predictions", scope=Scope.PIPELINE)

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast,
                                      input=model_cfg,
                                      output=predictions_cfg)

### Data Nodes and Task for result

<img src="img/result.svg" alt="drawing" width="500"/>

In [43]:
result_cfg = Config.configure_data_node(id="result", scope=Scope.PIPELINE)

task_result_cfg = Config.configure_task(id="task_result",
                                      function=result,
                                      input=[final_data_cfg, predictions_cfg, date_cfg],
                                      output=result_cfg)

## [Configuration of pipelines](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [44]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg])

pipeline_arima_cfg = Config.configure_pipeline(id="ARIMA",
                                               task_configs=[task_train_cfg,
                                                             task_forecast_cfg,
                                                             task_result_cfg])

## Add more models

<img src="img/pipeline_linear_regression.svg" alt="drawing" width="500"/>

In [45]:
def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return pd.Series(predictions)


task_train_linear_cfg = Config.configure_task(id="task_train_linear",
                                      function=train_linear_regression,
                                      input=train_data_cfg,
                                      output=model_cfg)

task_forecast_linear_cfg = Config.configure_task(id="task_forecast_linear",
                                      function=forecast_linear_regression,
                                      input=[model_cfg, date_cfg],
                                      output=predictions_cfg)

pipeline_linear_regression_cfg = Config.configure_pipeline(id="LinearRegression",
                                               task_configs=[task_train_linear_cfg,
                                                             task_forecast_linear_cfg,
                                                             task_result_cfg])

## [Configuration of scénario](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [46]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                          pipeline_arima_cfg,
                                                                          pipeline_linear_regression_cfg])

## Creation and submit of scenario

In [47]:
tp.Core().run()

[2023-03-24 22:51:57,279][Taipy][INFO] Issue(level='INFO', field='comparators', value=defaultdict(<class 'list'>, {}), message='No scenario comparators defined for scenario scenario', tag='_ScenarioConfigChecker')
[2023-03-24 22:51:57,289][Taipy][INFO] Development mode: Clean all entities of version 4a5046db-25a3-41ce-b1d6-3a8f5165a7f0


In [48]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

[2023-03-24 22:52:00,045][Taipy][INFO] job JOB_task_preprocess_data_81bded20-a65a-42b0-902d-ed7506009747 is completed.
[2023-03-24 22:52:00,062][Taipy][WARNING] DATANODE_model_c62e6f63-5cf4-4316-9cd5-9c35f8cf6678 cannot be read because it has never been written. Hint: The data node may refer to a wrong path : .data/pickles/DATANODE_model_c62e6f63-5cf4-4316-9cd5-9c35f8cf6678.p 
[2023-03-24 22:52:00,073][Taipy][WARNING] DATANODE_predictions_825c0591-f00c-4bcd-9400-0a3486fb56b0 cannot be read because it has never been written. Hint: The data node may refer to a wrong path : .data/pickles/DATANODE_predictions_825c0591-f00c-4bcd-9400-0a3486fb56b0.p 


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3403.257, Time=0.22 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3834.545, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3555.070, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3711.502, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3992.428, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3395.911, Time=0.22 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3469.260, Time=0.04 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3398.015, Time=0.34 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3395.735, Time=0.32 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3395.219, Time=0.26 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3639.930, Time=0.11 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3397.029, Time=0.33 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=3595.496, Time=0.14 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.40 sec
 ARIMA(1,1,2)(0,0,0)[0]             : 

{'PIPELINE_pipeline_preprocessing_2052efda-4499-4b05-9ff5-fe291cf927f1': [<taipy.core.job.job.Job at 0x7f71c60266e0>],
 'PIPELINE_ARIMA_10f3636e-88b5-431e-8d1b-ab9d45ed2c1e': [<taipy.core.job.job.Job at 0x7f71c5f63d60>,
 'PIPELINE_LinearRegression_7b6811f1-678e-4fd7-be4d-b2a57aed25d5': [<taipy.core.job.job.Job at 0x7f71c60264a0>,
  <taipy.core.job.job.Job at 0x7f71c6ff6710>]}

In [49]:
scenario.initial_data.read()

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
0,NaN,NaN,NaN,NaN,51526.0,41727.0,2191.0,2021-01-01
1,NaN,NaN,NaN,NaN,58316.0,33634.0,1181.0,2021-01-01
2,NaN,NaN,NaN,NaN,99897.0,67395.0,2762.0,2021-01-01
3,NaN,NaN,NaN,NaN,8117.0,7463.0,84.0,2021-01-01
4,NaN,NaN,NaN,NaN,17568.0,11146.0,405.0,2021-01-01
...,...,...,...,...,...,...,...,...
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31


In [50]:
scenario.train_data.read()

,Date,Deaths,Months,Days,Week,Day of week
0,2020-01-30,0.0,1,4,5,3
1,2020-01-31,0.0,1,5,5,4
2,2020-02-01,0.0,2,6,5,5
3,2020-02-02,0.0,2,7,5,6
4,2020-02-03,0.0,2,1,6,0
...,...,...,...,...,...,...
250,2020-10-06,104555.0,10,2,41,1
251,2020-10-07,105526.0,10,3,41,2
252,2020-10-08,106490.0,10,4,41,3
253,2020-10-09,107416.0,10,5,41,4


In [51]:
scenario.ARIMA.predictions.read()

array([109313.30106077, 110278.58254419, 111243.8357433 , 112209.06065893,
       113174.25729191, 114139.42564306, 115104.56571323, 116069.67750323,
       117034.76101389, 117999.81624605, 118964.84320053, 119929.84187816,
       120894.81227976, 121859.75440618, 122824.66825822, 123789.55383674,
       124754.41114254, 125719.24017647, 126684.04093934, 127648.813432  ,
       128613.55765526, 129578.27360995, 130542.96129691, 131507.62071695,
       132472.25187092, 133436.85475963, 134401.42938392, 135365.97574462,
       136330.49384254, 137294.98367853, 138259.4452534 , 139223.87856799,
       140188.28362313, 141152.66041963, 142117.00895834, 143081.32924008,
       144045.62126567, 145009.88503595, 145974.12055174, 146938.32781387,
       147902.50682317, 148866.65758046, 149830.78008658, 150794.87434235,
       151758.9403486 , 152722.97810616, 153686.98761585, 154650.9688785 ,
       155614.92189495, 156578.84666601, 157542.74319252, 158506.6114753 ,
       159470.45151518, 1

## Caching
Some job are skipped because no change has been done to the "input" Data Nodes.

In [52]:
tp.submit(scenario)

[2023-03-24 22:52:23,637][Taipy][INFO] job JOB_task_preprocess_data_afcdaa5a-6470-47c9-8fc6-588f9a3b097b is skipped.
[2023-03-24 22:52:23,684][Taipy][INFO] job JOB_task_train_5ad43019-c185-4920-b0af-5c425d5e7108 is skipped.
[2023-03-24 22:52:23,717][Taipy][INFO] job JOB_task_forecast_8172ac33-d0a7-4e6f-b481-fc4a4da0f6ba is completed.
[2023-03-24 22:52:23,742][Taipy][INFO] job JOB_task_result_04a62388-090e-4417-a569-71e7ad4ad50c is completed.
[2023-03-24 22:52:23,793][Taipy][INFO] job JOB_task_train_linear_b0706d1f-2961-408e-b1c8-1b6e877da831 is skipped.
[2023-03-24 22:52:23,827][Taipy][INFO] job JOB_task_forecast_linear_1a8c2dbc-cb15-48fc-85bb-199c9d5245e6 is completed.
[2023-03-24 22:52:23,853][Taipy][INFO] job JOB_task_result_9b559011-7ce0-46cc-afd6-7848b280a192 is completed.


{'PIPELINE_pipeline_preprocessing_2052efda-4499-4b05-9ff5-fe291cf927f1': [<taipy.core.job.job.Job at 0x7f71c70cf070>],
 'PIPELINE_ARIMA_10f3636e-88b5-431e-8d1b-ab9d45ed2c1e': [<taipy.core.job.job.Job at 0x7f71c6ff5ab0>,
 'PIPELINE_LinearRegression_7b6811f1-678e-4fd7-be4d-b2a57aed25d5': [<taipy.core.job.job.Job at 0x7f71c6be1f60>,
  <taipy.core.job.job.Job at 0x7f71c6be3fd0>]}

## Write in data nodes

To write a data node:

`<Data Node>.write(new_value)`

In [53]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

[2023-03-24 22:52:27,054][Taipy][INFO] job JOB_task_preprocess_data_23afd424-5ccf-48fc-b86a-f9e68d7fe0f0 is completed.


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3864.276, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4187.713, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3862.617, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4017.317, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4407.771, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3864.226, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3866.190, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3873.251, Time=0.01 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0] intercept
Total fit time: 0.246 seconds
[2023-03-24 22:52:27,382][Taipy][INFO] job JOB_task_train_beebefaa-9f02-40b0-89ef-eaf06a9969fc is completed.
[2023-03-24 22:52:27,416][Taipy][INFO] job JOB_task_forecast_3c093eea-676e-4697-b818-6545bd79d68b is completed.
[2023-03-24 22:52:27,442][Taipy][INFO] job JOB_task_result_aa3daeb7-7984-480f-a63e-a61ccc23d35c is completed.
[2023-03-24 22:52:27,

,Date,Deaths,Months,Days,Week,Day of week,Predictions
0,2020-01-22,0.0,1,3,4,2,NaN
1,2020-01-23,0.0,1,4,4,3,NaN
2,2020-01-24,0.0,1,5,4,4,NaN
3,2020-01-25,0.0,1,6,4,5,NaN
4,2020-01-26,0.0,1,7,4,6,NaN
...,...,...,...,...,...,...,...
340,2020-12-27,334533.0,12,7,52,6,NaN
341,2020-12-28,336438.0,12,1,53,0,NaN
342,2020-12-29,340061.0,12,2,53,1,NaN
343,2020-12-30,343783.0,12,3,53,2,NaN


In [54]:
scenario.ARIMA.predictions.read()

array([215495.10202493, 216178.19525252, 216883.98253204, 217608.85084325,
       218349.76237896, 219104.16296793, 219869.90507729, 220645.18307343,
       221428.47878973, 222218.51576021, 223014.22073943, 223814.69134863,
       224619.16887281, 225427.01538865, 226237.6945338 , 227050.7553378 ,
       227865.81862722, 228682.56559514, 229500.72819052, 230320.08103754,
       231140.43464145, 231961.62967617, 232783.53218118, 233606.02952325,
       234429.02700099, 235252.44499   , 236076.21654252, 236900.28536926,
       237724.60414251, 238549.13306937, 239373.83869222, 240198.69288006,
       241023.67198051, 241848.75610679, 242673.92853819, 243499.17521605,
       244324.48431991, 245149.8459112 , 245975.25163364, 246800.69446129,
       247626.16848679, 248451.66874328, 249277.19105463, 250102.73190959,
       250928.28835591, 251753.85791135, 252579.43848888, 253405.02833371,
       254230.62597044, 255056.23015855, 255881.83985503, 256707.45418292,
       257533.07240486, 2

## Simple framework

In [55]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

[2023-03-24 22:52:40,125][Taipy][INFO] job JOB_task_preprocess_data_69a5e3a1-7df7-4ff7-abb9-86ac476dfb2b is completed.
[2023-03-24 22:52:40,144][Taipy][WARNING] DATANODE_model_a8bc2d0a-4ef3-47e6-ae10-5246bec8e04c cannot be read because it has never been written. Hint: The data node may refer to a wrong path : .data/pickles/DATANODE_model_a8bc2d0a-4ef3-47e6-ae10-5246bec8e04c.p 
[2023-03-24 22:52:40,155][Taipy][WARNING] DATANODE_predictions_3cbeea9b-3ebe-4f70-81ac-67a4baf69464 cannot be read because it has never been written. Hint: The data node may refer to a wrong path : .data/pickles/DATANODE_predictions_3cbeea9b-3ebe-4f70-81ac-67a4baf69464.p 


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3403.257, Time=0.23 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3834.545, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3555.070, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3711.502, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3992.428, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3395.911, Time=0.21 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3469.260, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3398.015, Time=0.33 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3395.735, Time=0.30 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3395.219, Time=0.24 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3639.930, Time=0.09 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3397.029, Time=0.31 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=3595.496, Time=0.12 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(1,1,2)(0,0,0)[0]             : 

{'PIPELINE_pipeline_preprocessing_999c041b-4b10-46e5-8b54-ae0c31d83bf9': [<taipy.core.job.job.Job at 0x7f71c6f9c9d0>],
 'PIPELINE_ARIMA_9d71f6fd-279f-4499-95bd-7aadd508edbf': [<taipy.core.job.job.Job at 0x7f71c6b00550>,
 'PIPELINE_LinearRegression_1f0387c0-2e17-4633-a72b-1c7932520947': [<taipy.core.job.job.Job at 0x7f71c6a941f0>,
  <taipy.core.job.job.Job at 0x7f71c6ab7520>]}

In [56]:
scenario.ARIMA.task_forecast.function

<function __main__.forecast(model)>

In [57]:
scenario.ARIMA.model.read()

ARIMA(order=(1, 1, 2), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [58]:
scenario.pipelines['LinearRegression'].model.read()

LinearRegression()

In [59]:
[s.country.read() for s in tp.get_scenarios()]

['India', 'US']

In [60]:
[s.date.read() for s in tp.get_scenarios()]

[datetime.datetime(2020, 10, 10, 0, 0), datetime.datetime(2020, 10, 10, 0, 0)]

## Create a Gui for the backend
_scenario_selector_ lets you choose a scenario and display its results.

In [61]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

[('SCENARIO_scenario_b46f801e-e091-406a-93a0-7044e952557b', 'Second Scenario'), ('SCENARIO_scenario_eab14dc6-4dfc-47cd-a7f6-437d2ea65c80', 'First Scenario')] 
 SCENARIO_scenario_b46f801e-e091-406a-93a0-7044e952557b


In [62]:
result_arima = scenario.pipelines['ARIMA'].result.read()
result_rd = scenario.pipelines['LinearRegression'].result.read()
result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
result

,Date,Deaths_x,Months_x,Days_x,Week_x,Day of week_x,Predictions_x,Deaths_y,Months_y,Days_y,Week_y,Day of week_y,Predictions_y
0,2020-01-30,0.0,1,4,5,3,NaN,0.0,1,4,5,3,NaN
1,2020-01-31,0.0,1,5,5,4,NaN,0.0,1,5,5,4,NaN
2,2020-02-01,0.0,2,6,5,5,NaN,0.0,2,6,5,5,NaN
3,2020-02-02,0.0,2,7,5,6,NaN,0.0,2,7,5,6,NaN
4,2020-02-03,0.0,2,1,6,0,NaN,0.0,2,1,6,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,2020-12-27,147901.0,12,7,52,6,NaN,147901.0,12,7,52,6,NaN
333,2020-12-28,148153.0,12,1,53,0,NaN,148153.0,12,1,53,0,NaN
334,2020-12-29,148439.0,12,2,53,1,NaN,148439.0,12,2,53,1,NaN
335,2020-12-30,148738.0,12,3,53,2,NaN,148738.0,12,3,53,2,NaN


**Tips** : the _value_by_id_ property if set to True for a selected will make _selected_scenario_ directly refer to the first element of the tupple (here the id)

![](img/predictions.png)

In [63]:
prediction_md = """
<|layout|columns=1 2 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

Prediction date
<|{first_date}|date|>

<|{selected_country}|selector|lov={country_lov}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths_x|type[1]=bar|y[2]=Predictions_x|y[3]=Predictions_y|>
"""

In [64]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [65]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    scenario = tp.get(state.selected_scenario)
    scenario.country.write(state.selected_country)
    scenario.date.write(state.first_date.replace(tzinfo=None))
    tp.submit(scenario)
    actualize_graph(state)

In [66]:
def actualize_graph(state):
    # 1) update the result dataframe
    # 2) change selected_country with the predicted country of the scenario
    scenario = tp.get(state.selected_scenario)
    result_arima = scenario.pipelines['ARIMA'].result.read()
    result_rd = scenario.pipelines['LinearRegression'].result.read()
    if result_arima is not None and result_rd is not None:
        state.result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
    state.selected_country = scenario.country.read()

In [67]:
Gui(prediction_md).run(dark_mode=False, run_browser=False)

[2023-03-24 22:52:56,399][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:52:56,452][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:52:56,453][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


# Multi-pages and Taipy Rest

To create a multi-pages app, we only need a dictionary with names as the keys and the Markdowns as the values.

The _navbar_ control (<|navbar|>) has a default behaviour. It redirects to the different pages of the app automatically. Other solutions exists.

![](img/multi_pages.png)

In [68]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
gui_multi_pages.run(dark_mode=False, run_browser=False)
# tp.run(gui_multi_pages, rest, dark_mode=False, run_browser=False)

[2023-03-24 22:52:59,215][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2023-03-24 22:52:59,281][Taipy][INFO] Gui server has been stopped.
[2023-03-24 22:52:59,282][Taipy][INFO]  * Server starting on http://127.0.0.1:5000
